# Create Query for Organization

In [4]:
import sqlite3
import pandas as pd

con = sqlite3.connect('cards.db')
cur = con.cursor()

In [5]:
# load dataset
df = pd.read_csv("./data/cards.csv")

# rename columns
df = df.rename(columns={
    "index": "class_index",
    "type": "card_type"
})

# drop columns
df = df.drop(columns=["Unnamed: 0"])

df.head()


,class_index,file_path,label,card_type,dataset,rank,suit,rank_num,brightness
0,0,./data/train/ace of clubs/001.jpg,ace of clubs,ace,train,ace,clubs,1,195.890286
1,0,./data/train/ace of clubs/002.jpg,ace of clubs,ace,train,ace,clubs,1,213.886599
2,0,./data/train/ace of clubs/003.jpg,ace of clubs,ace,train,ace,clubs,1,205.301160
3,0,./data/train/ace of clubs/004.jpg,ace of clubs,ace,train,ace,clubs,1,198.171855
4,0,./data/train/ace of clubs/005.jpg,ace of clubs,ace,train,ace,clubs,1,167.438716


In [6]:
# create table
cur.execute("""
CREATE TABLE IF NOT EXISTS cards (
    id          INTEGER PRIMARY KEY AUTOINCREMENT,
    class_index INTEGER,
    file_path   TEXT NOT NULL,
    label       TEXT NOT NULL,
    card_type   TEXT,
    dataset     TEXT,
    rank        TEXT,
    suit        TEXT,
    rank_num    INTEGER,
    brightness  REAL
);
""")

# insert rows
records = df.to_dict(orient="records")

cur.executemany("""
INSERT INTO cards (
    class_index, file_path, label, card_type, dataset, 
    rank, suit, rank_num, brightness
) VALUES (
    :class_index, :file_path, :label, :card_type, :dataset, 
    :rank, :suit, :rank_num, :brightness
);
""", records)

### Quick look at database

In [9]:
# suit distribution (train, test, valid)
query = """
    SELECT
        suit,
        COUNT(*) AS n_images
    FROM cards
    WHERE dataset = 'train'
    GROUP BY suit
    ORDER BY n_images DESC;
"""

# execute query and get results
cur.execute(query)
rows = cur.fetchall()

print(rows)


[('spades', 4062), ('hearts', 3690), ('diamonds', 3654), ('clubs', 3614), ('joker', 230)]


In [10]:
con.close()